In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('once')
from sklearn.tree import DecisionTreeRegressor
import datetime as dt

In [2]:
df = pd.read_csv('../datos/cnt/11-cnt-cod_est.csv', index_col=0)
df.head(2)

,atemp,hum,windspeed,season,yr,holiday,weekday,weathersit,cnt,x0_1,...,x0_3,x0_4,x0_5,x0_6,x0_7,x0_8,x0_9,x0_10,x0_11,x0_12
0,-0.456399,0.855582,-0.208909,0,0,0,0,1,985,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.492863,0.332722,0.687550,0,0,2,0,1,801,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df.dtypes

atemp         float64
hum           float64
windspeed     float64
season          int64
yr              int64
holiday         int64
weekday         int64
weathersit      int64
cnt             int64
x0_1          float64
x0_2          float64
x0_3          float64
x0_4          float64
x0_5          float64
x0_6          float64
x0_7          float64
x0_8          float64
x0_9          float64
x0_10         float64
x0_11         float64
x0_12         float64
dtype: object

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 729
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   atemp       730 non-null    float64
 1   hum         730 non-null    float64
 2   windspeed   730 non-null    float64
 3   season      730 non-null    int64  
 4   yr          730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   weathersit  730 non-null    int64  
 8   cnt         730 non-null    int64  
 9   x0_1        730 non-null    float64
 10  x0_2        730 non-null    float64
 11  x0_3        730 non-null    float64
 12  x0_4        730 non-null    float64
 13  x0_5        730 non-null    float64
 14  x0_6        730 non-null    float64
 15  x0_7        730 non-null    float64
 16  x0_8        730 non-null    float64
 17  x0_9        730 non-null    float64
 18  x0_10       730 non-null    float64
 19  x0_11       730 non-null    f

In [5]:
df.head(2)

,atemp,hum,windspeed,season,yr,holiday,weekday,weathersit,cnt,x0_1,...,x0_3,x0_4,x0_5,x0_6,x0_7,x0_8,x0_9,x0_10,x0_11,x0_12
0,-0.456399,0.855582,-0.208909,0,0,0,0,1,985,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.492863,0.332722,0.687550,0,0,2,0,1,801,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---

In [6]:
X = df.drop("cnt", axis = 1)
y = df["cnt"]

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42) 

In [8]:
arbol = DecisionTreeRegressor(random_state =0)
arbol.fit(x_train, y_train)

DecisionTreeRegressor(random_state=0)

In [24]:
#fig = plt.figure(figsize = (10,6))
#tree.plot_tree(arbol, feature_names = x_train.columns, filled = True)
#plt.show()

In [9]:
max_features = np.sqrt(len(x_train.columns))
max_features

4.47213595499958

In [10]:
print(arbol.tree_.max_depth)

28


In [11]:
param = {"max_depth": [4,6,8,10,12],
        "max_features": [1,2,3,4],
        "min_samples_split": [10,40,70],
        "min_samples_leaf": [10,40,70]} 

In [12]:
gs = GridSearchCV(
            estimator=DecisionTreeRegressor(), 
            param_grid= param, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error")
gs.fit(x_train, y_train)



GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [4, 6, 8, 10, 12],
                         'max_features': [1, 2, 3, 4],
                         'min_samples_leaf': [10, 40, 70],
                         'min_samples_split': [10, 40, 70]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [13]:
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=8, max_features=4, min_samples_leaf=10,
                      min_samples_split=10)

In [30]:
#fig = plt.figure(figsize=(40, 20))
#tree.plot_tree(mejor_modelo, feature_names=x_train.columns, filled=True);

In [14]:
y_pred_test_dt2 = mejor_modelo.predict(x_test)
y_pred_train_dt2 = mejor_modelo.predict(x_train)

In [15]:

def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [16]:
dt_results1 = metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, "1_Dtree_CNT_est-cod")
dt_results1

,MAE,MSE,RMSE,R2,set,modelo
0,1104.437166,1.801065e+06,1342.037690,0.473810,test,1_Dtree_CNT_est-cod
1,982.461049,1.495148e+06,1222.762621,0.608158,train,1_Dtree_CNT_est-cod


Random forest

In [17]:
X = df.drop("cnt", axis = 1)
y = df["cnt"]

In [18]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [19]:
param = {"max_depth": [4,6,8,10,12],
        "max_features": [1,2,3,4],
        "min_samples_split": [10,40,70],
        "min_samples_leaf": [10,40,70]} 

In [20]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(), 
            param_grid= param, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 


In [21]:
gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [4, 6, 8, 10, 12],
                         'max_features': [1, 2, 3, 4],
                         'min_samples_leaf': [10, 40, 70],
                         'min_samples_split': [10, 40, 70]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [22]:
bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=8, max_features=4, min_samples_leaf=10,
                      min_samples_split=10)

In [40]:
'''for arbol in tqdm(bosque.estimators_):
    fig = plt.figure(figsize=(40, 20))
    tree.plot_tree(arbol, feature_names= x_train.columns, filled = True)
    plt.show()'''

'for arbol in tqdm(bosque.estimators_):\n    fig = plt.figure(figsize=(40, 20))\n    tree.plot_tree(arbol, feature_names= x_train.columns, filled = True)\n    plt.show()'

In [23]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
   
   
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df


In [24]:
dt_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "2_RForest_CNT_cod_estand")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,616.029975,584679.193544,764.643181,0.829183,test,2_RForest_CNT_cod_estand
1,617.336373,646969.323343,804.344033,0.830445,train,2_RForest_CNT_cod_estand


In [25]:
df_todos_resultados = pd.concat([dt_results, dt_results1], axis = 0)
df_todos_resultados


,MAE,MSE,RMSE,R2,set,modelo
0,616.029975,5.846792e+05,764.643181,0.829183,test,2_RForest_CNT_cod_estand
1,617.336373,6.469693e+05,804.344033,0.830445,train,2_RForest_CNT_cod_estand
0,1104.437166,1.801065e+06,1342.037690,0.473810,test,1_Dtree_CNT_est-cod
1,982.461049,1.495148e+06,1222.762621,0.608158,train,1_Dtree_CNT_est-cod


In [44]:
df_todos_resultados.to_csv('../datos/cnt/12-cnt-metricas-estand.csv')

# Random Forest es la mejor opcion